In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1D CNN(Predict_LOG)

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

csv_file = "/content/drive/MyDrive/HAR Data & Models/sb_IMU.csv"
model_path = "/content/drive/MyDrive/HAR Data & Models/wisdm_final_1dcnn_model.h5"
output_log = "1D_CNN.csv"

window_size = 60
stride = 60
sampling_rate = 30
features = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
label_column = "Label"

categories = [1, 2, 3, 4, 5]

def decode_activity(label):
    mapping = {
        1: "Walking",
        2: "Jogging",
        3: "Sitting",
        4: "Standing",
        5: "Stair "
    }
    return mapping.get(label, "Unknown")


def normalize_segment(segment_array):
    scaler = StandardScaler()
    normalized = scaler.fit_transform(segment_array)  # shape (60, 6)
    return normalized


if not os.path.exists(output_log):
    with open(output_log, "w") as f:
        f.write("Timestamp,Actual_Label,Predicted_Label,Latency(ms),Valid\n")


model = load_model(model_path)

def live_predict():
    print("📡 Monitoring for new data in sliding windows...\nPress Ctrl+C to stop.\n")
    last_row_read = 0

    all_actual = []
    all_predicted = []

    try:
        while True:
            if not os.path.exists(csv_file):
                print(f"Waiting for file {csv_file} to appear...")
                time.sleep(1)
                continue

            df = pd.read_csv(csv_file,low_memory=False)

            if not all(col in df.columns for col in features + [label_column]):
                print("Missing required columns. Waiting...")
                time.sleep(1)
                continue

            total_rows = len(df)

            if total_rows >= window_size:
                while last_row_read + window_size <= total_rows:
                    window = df.iloc[last_row_read:last_row_read + window_size]
                    actual_labels = window[label_column].values

                    if np.all(actual_labels == 0):
                        print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → Skipped (Label 0)")
                        last_row_read += stride
                        continue

                    feature_chunk = window[features].values
                    norm_chunk = normalize_segment(feature_chunk)
                    X_input = norm_chunk[np.newaxis, ...]

                    start_time = time.time()
                    prediction = model.predict(X_input, verbose=0)[0]
                    latency_ms = round((time.time() - start_time) * 1000, 2)

                    predicted_label = categories[np.argmax(prediction)]
                    decoded_pred = decode_activity(predicted_label)

                    actual_labels = actual_labels[actual_labels != 0]
                    actual_label = np.bincount(actual_labels).argmax() if len(actual_labels) > 0 else 0
                    decoded_actual = decode_activity(actual_label)

                    # Store results for accuracy later
                    if actual_label != 0:
                        all_actual.append(actual_label)
                        all_predicted.append(predicted_label)

                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    match = "✅" if predicted_label == actual_label else "❌"

                    print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → "
                          f"Predicted: {decoded_pred} ({predicted_label}) | "
                          f"Actual: {decoded_actual} ({actual_label}) {match} | "
                          f"Latency: {latency_ms} ms")

                    with open(output_log, "a") as f:
                        f.write(f"{timestamp},{decoded_actual},{decoded_pred},{latency_ms},{int(predicted_label == actual_label)}\n")

                    last_row_read += stride

            time.sleep(1 / sampling_rate)

    except KeyboardInterrupt:
        print("\n⛔ Stopped by user.")
        if all_actual:
            acc = accuracy_score(all_actual, all_predicted) * 100
            print(f"\n Final Accuracy: {acc:.2f}% "
                  f"({len(all_actual)} evaluated windows)")
        else:
            print("\n⚠️ No valid windows evaluated. Accuracy cannot be computed.")


if __name__ == "__main__":
    live_predict()


📡 Monitoring for new data in sliding windows...
Press Ctrl+C to stop.

🪟 Rows 1-60 → Skipped (Label 0)
🪟 Rows 61-120 → Skipped (Label 0)
🪟 Rows 121-180 → Skipped (Label 0)
🪟 Rows 181-240 → Skipped (Label 0)
🪟 Rows 241-300 → Skipped (Label 0)
🪟 Rows 301-360 → Skipped (Label 0)
🪟 Rows 361-420 → Skipped (Label 0)
🪟 Rows 421-480 → Skipped (Label 0)
🪟 Rows 481-540 → Skipped (Label 0)
🪟 Rows 541-600 → Skipped (Label 0)
🪟 Rows 601-660 → Skipped (Label 0)
🪟 Rows 661-720 → Skipped (Label 0)
🪟 Rows 721-780 → Skipped (Label 0)
🪟 Rows 781-840 → Skipped (Label 0)
🪟 Rows 841-900 → Skipped (Label 0)
🪟 Rows 901-960 → Skipped (Label 0)
🪟 Rows 961-1020 → Skipped (Label 0)
🪟 Rows 1021-1080 → Skipped (Label 0)
🪟 Rows 1081-1140 → Skipped (Label 0)
🪟 Rows 1141-1200 → Skipped (Label 0)
🪟 Rows 1201-1260 → Skipped (Label 0)
🪟 Rows 1261-1320 → Skipped (Label 0)
🪟 Rows 1321-1380 → Skipped (Label 0)
🪟 Rows 1381-1440 → Skipped (Label 0)
🪟 Rows 1441-1500 → Skipped (Label 0)
🪟 Rows 1501-1560 → Skipped (Label 0)
🪟 Ro

#LSTM (Predict_LOG)

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model


csv_file = "/content/drive/MyDrive/HAR Data & Models/sb_IMU.csv"
model_path = "/content/drive/MyDrive/HAR Data & Models/best_lstm_model.h5"
output_log = "LSTM.csv"

window_size = 60
stride = 60
sampling_rate = 30
features = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
label_column = "Label"

categories = [1, 2, 3, 4, 5, 6]

def decode_activity(label):
    mapping = {
        1: "Walking",
        2: "Running",
        3: "Sitting",
        4: "Standing",
        5: "Stair Down",
        6: "Stair Up"
    }
    return mapping.get(label, "Unknown")


def normalize_segment(segment_array):
    """
    Normalize each feature (column) across the 60 timesteps.
    Keeps the shape (60, 6) for Conv1D.
    """
    scaler = StandardScaler()
    normalized = scaler.fit_transform(segment_array)  # shape (60, 6)
    return normalized


if not os.path.exists(output_log):
    with open(output_log, "w") as f:
        f.write("Timestamp,Actual_Label,Predicted_Label,Latency(ms),Valid\n")


model = load_model(model_path)

def live_predict():
    print("📡 Monitoring for new data in sliding windows...\nPress Ctrl+C to stop.\n")
    last_row_read = 0

    try:
        while True:
            if not os.path.exists(csv_file):
                print(f"Waiting for file {csv_file} to appear...")
                time.sleep(1)
                continue

            df = pd.read_csv(csv_file)

            # Ensure necessary columns exist
            if not all(col in df.columns for col in features + [label_column]):
                print("Missing required columns. Waiting...")
                time.sleep(1)
                continue

            total_rows = len(df)

            if total_rows >= window_size:
                while last_row_read + window_size <= total_rows:
                    window = df.iloc[last_row_read:last_row_read + window_size]
                    actual_labels = window[label_column].values

                    # Skip windows with all labels = 0
                    if np.all(actual_labels == 0):
                        print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → Skipped (Label 0)")
                        last_row_read += stride
                        continue

                    # Apply preprocessing
                    feature_chunk = window[features].values  # (60, 6)
                    norm_chunk = normalize_segment(feature_chunk)  # (60, 6)

                    # Model expects (1, 60, 6)
                    X_input = norm_chunk[np.newaxis, ...]

                    # Prediction
                    start_time = time.time()
                    prediction = model.predict(X_input, verbose=0)[0]
                    latency_ms = round((time.time() - start_time) * 1000, 2)

                    predicted_label = categories[np.argmax(prediction)]
                    decoded_pred = decode_activity(predicted_label)

                    # Actual label = majority vote in window
                    actual_labels = actual_labels[actual_labels != 0]
                    actual_label = np.bincount(actual_labels).argmax() if len(actual_labels) > 0 else 0
                    decoded_actual = decode_activity(actual_label)

                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    match = "✅" if predicted_label == actual_label else "❌"

                    print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → "
                          f"Predicted: {decoded_pred} ({predicted_label}) | "
                          f"Actual: {decoded_actual} ({actual_label}) {match} | "
                          f"Latency: {latency_ms} ms")

                    # Save log
                    with open(output_log, "a") as f:
                        f.write(f"{timestamp},{decoded_actual},{decoded_pred},{latency_ms},{int(predicted_label == actual_label)}\n")

                    last_row_read += stride

            time.sleep(1 / sampling_rate)

    except KeyboardInterrupt:
        print("\n⛔ Stopped by user.")

if __name__ == "__main__":
    live_predict()


📡 Monitoring for new data in sliding windows...
Press Ctrl+C to stop.

🪟 Rows 1-60 → Skipped (Label 0)
🪟 Rows 61-120 → Skipped (Label 0)
🪟 Rows 121-180 → Skipped (Label 0)
🪟 Rows 181-240 → Skipped (Label 0)
🪟 Rows 241-300 → Skipped (Label 0)
🪟 Rows 301-360 → Skipped (Label 0)
🪟 Rows 361-420 → Skipped (Label 0)
🪟 Rows 421-480 → Skipped (Label 0)
🪟 Rows 481-540 → Skipped (Label 0)
🪟 Rows 541-600 → Skipped (Label 0)
🪟 Rows 601-660 → Skipped (Label 0)
🪟 Rows 661-720 → Skipped (Label 0)
🪟 Rows 721-780 → Skipped (Label 0)
🪟 Rows 781-840 → Skipped (Label 0)
🪟 Rows 841-900 → Skipped (Label 0)
🪟 Rows 901-960 → Skipped (Label 0)
🪟 Rows 961-1020 → Skipped (Label 0)
🪟 Rows 1021-1080 → Skipped (Label 0)
🪟 Rows 1081-1140 → Skipped (Label 0)
🪟 Rows 1141-1200 → Skipped (Label 0)
🪟 Rows 1201-1260 → Skipped (Label 0)
🪟 Rows 1261-1320 → Skipped (Label 0)
🪟 Rows 1321-1380 → Skipped (Label 0)
🪟 Rows 1381-1440 → Skipped (Label 0)
🪟 Rows 1441-1500 → Skipped (Label 0)
🪟 Rows 1501-1560 → Skipped (Label 0)
🪟 Ro

#LSTM + 1D_CNN (Predict_LOG)

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model


csv_file = "/content/drive/MyDrive/HAR Data & Models/sb_IMU.csv"
model_path = "/content/drive/MyDrive/HAR Data & Models/best_cnn_lstm_model.h5"
output_log = "LSTM_CNN.csv"

window_size = 60
stride = 60
sampling_rate = 30
features = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
label_column = "Label"

categories = [1, 2, 3, 4, 5, 6]

def decode_activity(label):
    mapping = {
        1: "Walking",
        2: "Running",
        3: "Sitting",
        4: "Standing",
        5: "Stair Down",
        6: "Stair Up"
    }
    return mapping.get(label, "Unknown")


def normalize_segment(segment_array):
    """
    Normalize each feature (column) across the 60 timesteps.
    Keeps the shape (60, 6) for Conv1D.
    """
    scaler = StandardScaler()
    normalized = scaler.fit_transform(segment_array)  # shape (60, 6)
    return normalized


if not os.path.exists(output_log):
    with open(output_log, "w") as f:
        f.write("Timestamp,Actual_Label,Predicted_Label,Latency(ms),Valid\n")


model = load_model(model_path)

def live_predict():
    print("📡 Monitoring for new data in sliding windows...\nPress Ctrl+C to stop.\n")
    last_row_read = 0

    try:
        while True:
            if not os.path.exists(csv_file):
                print(f"Waiting for file {csv_file} to appear...")
                time.sleep(1)
                continue

            df = pd.read_csv(csv_file)

            # Ensure necessary columns exist
            if not all(col in df.columns for col in features + [label_column]):
                print("Missing required columns. Waiting...")
                time.sleep(1)
                continue

            total_rows = len(df)

            if total_rows >= window_size:
                while last_row_read + window_size <= total_rows:
                    window = df.iloc[last_row_read:last_row_read + window_size]
                    actual_labels = window[label_column].values

                    # Skip windows with all labels = 0
                    if np.all(actual_labels == 0):
                        print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → Skipped (Label 0)")
                        last_row_read += stride
                        continue

                    # Apply preprocessing
                    feature_chunk = window[features].values  # (60, 6)
                    norm_chunk = normalize_segment(feature_chunk)  # (60, 6)

                    # Model expects (1, 60, 6)
                    X_input = norm_chunk[np.newaxis, ...]

                    # Prediction
                    start_time = time.time()
                    prediction = model.predict(X_input, verbose=0)[0]
                    latency_ms = round((time.time() - start_time) * 1000, 2)

                    predicted_label = categories[np.argmax(prediction)]
                    decoded_pred = decode_activity(predicted_label)

                    # Actual label = majority vote in window
                    actual_labels = actual_labels[actual_labels != 0]
                    actual_label = np.bincount(actual_labels).argmax() if len(actual_labels) > 0 else 0
                    decoded_actual = decode_activity(actual_label)

                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    match = "✅" if predicted_label == actual_label else "❌"

                    print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → "
                          f"Predicted: {decoded_pred} ({predicted_label}) | "
                          f"Actual: {decoded_actual} ({actual_label}) {match} | "
                          f"Latency: {latency_ms} ms")

                    # Save log
                    with open(output_log, "a") as f:
                        f.write(f"{timestamp},{decoded_actual},{decoded_pred},{latency_ms},{int(predicted_label == actual_label)}\n")

                    last_row_read += stride

            time.sleep(1 / sampling_rate)

    except KeyboardInterrupt:
        print("\n⛔ Stopped by user.")

if __name__ == "__main__":
    live_predict()


📡 Monitoring for new data in sliding windows...
Press Ctrl+C to stop.

🪟 Rows 1-60 → Skipped (Label 0)
🪟 Rows 61-120 → Skipped (Label 0)
🪟 Rows 121-180 → Skipped (Label 0)
🪟 Rows 181-240 → Skipped (Label 0)
🪟 Rows 241-300 → Skipped (Label 0)
🪟 Rows 301-360 → Skipped (Label 0)
🪟 Rows 361-420 → Skipped (Label 0)
🪟 Rows 421-480 → Skipped (Label 0)
🪟 Rows 481-540 → Skipped (Label 0)
🪟 Rows 541-600 → Skipped (Label 0)
🪟 Rows 601-660 → Skipped (Label 0)
🪟 Rows 661-720 → Skipped (Label 0)
🪟 Rows 721-780 → Skipped (Label 0)
🪟 Rows 781-840 → Skipped (Label 0)
🪟 Rows 841-900 → Skipped (Label 0)
🪟 Rows 901-960 → Skipped (Label 0)
🪟 Rows 961-1020 → Skipped (Label 0)
🪟 Rows 1021-1080 → Skipped (Label 0)
🪟 Rows 1081-1140 → Skipped (Label 0)
🪟 Rows 1141-1200 → Skipped (Label 0)
🪟 Rows 1201-1260 → Skipped (Label 0)
🪟 Rows 1261-1320 → Skipped (Label 0)
🪟 Rows 1321-1380 → Skipped (Label 0)
🪟 Rows 1381-1440 → Skipped (Label 0)
🪟 Rows 1441-1500 → Skipped (Label 0)
🪟 Rows 1501-1560 → Skipped (Label 0)
🪟 Ro

#LSTM + 2D_CNN (Predict_LOG)

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model


csv_file = "/content/drive/MyDrive/HAR Data & Models/sb_IMU.csv"
model_path = "/content/drive/MyDrive/HAR Data & Models/best_cnn2d_lstm_model.h5"
output_log = "LSTM_2d_CNN.csv"

window_size = 60
stride = 60
sampling_rate = 30
features = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
label_column = "Label"

categories = [1, 2, 3, 4, 5, 6]

def decode_activity(label):
    mapping = {
        1: "Walking",
        2: "Running",
        3: "Sitting",
        4: "Standing",
        5: "Stair Down",
        6: "Stair Up"
    }
    return mapping.get(label, "Unknown")


def normalize_segment(segment_array):
    """
    Normalize each feature (column) across the 60 timesteps.
    Keeps the shape (60, 6) for Conv1D.
    """
    scaler = StandardScaler()
    normalized = scaler.fit_transform(segment_array)  # shape (60, 6)
    return normalized


if not os.path.exists(output_log):
    with open(output_log, "w") as f:
        f.write("Timestamp,Actual_Label,Predicted_Label,Latency(ms),Valid\n")


model = load_model(model_path)

def live_predict():
    print("📡 Monitoring for new data in sliding windows...\nPress Ctrl+C to stop.\n")
    last_row_read = 0

    try:
        while True:
            if not os.path.exists(csv_file):
                print(f"Waiting for file {csv_file} to appear...")
                time.sleep(1)
                continue

            df = pd.read_csv(csv_file)

            # Ensure necessary columns exist
            if not all(col in df.columns for col in features + [label_column]):
                print("Missing required columns. Waiting...")
                time.sleep(1)
                continue

            total_rows = len(df)

            if total_rows >= window_size:
                while last_row_read + window_size <= total_rows:
                    window = df.iloc[last_row_read:last_row_read + window_size]
                    actual_labels = window[label_column].values

                    # Skip windows with all labels = 0
                    if np.all(actual_labels == 0):
                        print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → Skipped (Label 0)")
                        last_row_read += stride
                        continue
                    # Apply preprocessing
                    feature_chunk = window[features].values  # (60, 6)
                    norm_chunk = normalize_segment(feature_chunk)  # (60, 6)

                    # Model expects (1, 60, 6)
                    X_input = norm_chunk[np.newaxis, ...]

                    # Prediction
                    start_time = time.time()
                    prediction = model.predict(X_input, verbose=0)[0]
                    latency_ms = round((time.time() - start_time) * 1000, 2)

                    predicted_label = categories[np.argmax(prediction)]
                    decoded_pred = decode_activity(predicted_label)

                    # Actual label = majority vote in window
                    actual_labels = actual_labels[actual_labels != 0]
                    actual_label = np.bincount(actual_labels).argmax() if len(actual_labels) > 0 else 0
                    decoded_actual = decode_activity(actual_label)

                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    match = "✅" if predicted_label == actual_label else "❌"

                    print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → "
                          f"Predicted: {decoded_pred} ({predicted_label}) | "
                          f"Actual: {decoded_actual} ({actual_label}) {match} | "
                          f"Latency: {latency_ms} ms")

                    # Save log
                    with open(output_log, "a") as f:
                        f.write(f"{timestamp},{decoded_actual},{decoded_pred},{latency_ms},{int(predicted_label == actual_label)}\n")

                    last_row_read += stride

            time.sleep(1 / sampling_rate)

    except KeyboardInterrupt:
        print("\n⛔ Stopped by user.")

if __name__ == "__main__":
    live_predict()


📡 Monitoring for new data in sliding windows...
Press Ctrl+C to stop.

🪟 Rows 1-60 → Skipped (Label 0)
🪟 Rows 61-120 → Skipped (Label 0)
🪟 Rows 121-180 → Skipped (Label 0)
🪟 Rows 181-240 → Skipped (Label 0)
🪟 Rows 241-300 → Skipped (Label 0)
🪟 Rows 301-360 → Skipped (Label 0)
🪟 Rows 361-420 → Skipped (Label 0)
🪟 Rows 421-480 → Skipped (Label 0)
🪟 Rows 481-540 → Skipped (Label 0)
🪟 Rows 541-600 → Skipped (Label 0)
🪟 Rows 601-660 → Skipped (Label 0)
🪟 Rows 661-720 → Skipped (Label 0)
🪟 Rows 721-780 → Skipped (Label 0)
🪟 Rows 781-840 → Skipped (Label 0)
🪟 Rows 841-900 → Skipped (Label 0)
🪟 Rows 901-960 → Skipped (Label 0)
🪟 Rows 961-1020 → Skipped (Label 0)
🪟 Rows 1021-1080 → Skipped (Label 0)
🪟 Rows 1081-1140 → Skipped (Label 0)
🪟 Rows 1141-1200 → Skipped (Label 0)
🪟 Rows 1201-1260 → Skipped (Label 0)
🪟 Rows 1261-1320 → Skipped (Label 0)
🪟 Rows 1321-1380 → Skipped (Label 0)
🪟 Rows 1381-1440 → Skipped (Label 0)
🪟 Rows 1441-1500 → Skipped (Label 0)
🪟 Rows 1501-1560 → Skipped (Label 0)
🪟 Ro

#2D CNN(Predict Log)

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model


csv_file = "/content/drive/MyDrive/HAR Data & Models/sb_IMU.csv"
model_path = "/content/drive/MyDrive/HAR Data & Models/best_2dcnn_model.h5"
output_log = "2d_CNN.csv"

window_size = 60
stride = 60
sampling_rate = 30
features = ['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
label_column = "Label"

categories = [1, 2, 3, 4, 5, 6]

def decode_activity(label):
    mapping = {
        1: "Walking",
        2: "Running",
        3: "Sitting",
        4: "Standing",
        5: "Stair Down",
        6: "Stair Up"
    }
    return mapping.get(label, "Unknown")


def normalize_segment(segment_array):
    """
    Normalize each feature (column) across the 60 timesteps.
    Keeps the shape (60, 6) for Conv1D.
    """
    scaler = StandardScaler()
    normalized = scaler.fit_transform(segment_array)  # shape (60, 6)
    return normalized


if not os.path.exists(output_log):
    with open(output_log, "w") as f:
        f.write("Timestamp,Actual_Label,Predicted_Label,Latency(ms),Valid\n")


model = load_model(model_path)

def live_predict():
    print("📡 Monitoring for new data in sliding windows...\nPress Ctrl+C to stop.\n")
    last_row_read = 0

    try:
        while True:
            if not os.path.exists(csv_file):
                print(f"Waiting for file {csv_file} to appear...")
                time.sleep(1)
                continue

            df = pd.read_csv(csv_file)

            # Ensure necessary columns exist
            if not all(col in df.columns for col in features + [label_column]):
                print("Missing required columns. Waiting...")
                time.sleep(1)
                continue

            total_rows = len(df)

            if total_rows >= window_size:
                while last_row_read + window_size <= total_rows:
                    window = df.iloc[last_row_read:last_row_read + window_size]
                    actual_labels = window[label_column].values

                    # Skip windows with all labels = 0
                    if np.all(actual_labels == 0):
                        print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → Skipped (Label 0)")
                        last_row_read += stride
                        continue
                    # Apply preprocessing
                    feature_chunk = window[features].values  # (60, 6)
                    norm_chunk = normalize_segment(feature_chunk)  # (60, 6)

                    # Model expects (1, 60, 6)
                    X_input = norm_chunk[np.newaxis, ...]

                    # Prediction
                    start_time = time.time()
                    prediction = model.predict(X_input, verbose=0)[0]
                    latency_ms = round((time.time() - start_time) * 1000, 2)

                    predicted_label = categories[np.argmax(prediction)]
                    decoded_pred = decode_activity(predicted_label)

                    # Actual label = majority vote in window
                    actual_labels = actual_labels[actual_labels != 0]
                    actual_label = np.bincount(actual_labels).argmax() if len(actual_labels) > 0 else 0
                    decoded_actual = decode_activity(actual_label)

                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    match = "✅" if predicted_label == actual_label else "❌"

                    print(f"🪟 Rows {last_row_read+1}-{last_row_read+window_size} → "
                          f"Predicted: {decoded_pred} ({predicted_label}) | "
                          f"Actual: {decoded_actual} ({actual_label}) {match} | "
                          f"Latency: {latency_ms} ms")

                    # Save log
                    with open(output_log, "a") as f:
                        f.write(f"{timestamp},{decoded_actual},{decoded_pred},{latency_ms},{int(predicted_label == actual_label)}\n")

                    last_row_read += stride

            time.sleep(1 / sampling_rate)

    except KeyboardInterrupt:
        print("\n⛔ Stopped by user.")

if __name__ == "__main__":
    live_predict()


📡 Monitoring for new data in sliding windows...
Press Ctrl+C to stop.

🪟 Rows 1-60 → Skipped (Label 0)
🪟 Rows 61-120 → Skipped (Label 0)
🪟 Rows 121-180 → Skipped (Label 0)
🪟 Rows 181-240 → Skipped (Label 0)
🪟 Rows 241-300 → Skipped (Label 0)
🪟 Rows 301-360 → Skipped (Label 0)
🪟 Rows 361-420 → Skipped (Label 0)
🪟 Rows 421-480 → Skipped (Label 0)
🪟 Rows 481-540 → Skipped (Label 0)
🪟 Rows 541-600 → Skipped (Label 0)
🪟 Rows 601-660 → Skipped (Label 0)
🪟 Rows 661-720 → Skipped (Label 0)
🪟 Rows 721-780 → Skipped (Label 0)
🪟 Rows 781-840 → Skipped (Label 0)
🪟 Rows 841-900 → Skipped (Label 0)
🪟 Rows 901-960 → Skipped (Label 0)
🪟 Rows 961-1020 → Skipped (Label 0)
🪟 Rows 1021-1080 → Skipped (Label 0)
🪟 Rows 1081-1140 → Skipped (Label 0)
🪟 Rows 1141-1200 → Skipped (Label 0)
🪟 Rows 1201-1260 → Skipped (Label 0)
🪟 Rows 1261-1320 → Skipped (Label 0)
🪟 Rows 1321-1380 → Skipped (Label 0)
🪟 Rows 1381-1440 → Skipped (Label 0)
🪟 Rows 1441-1500 → Skipped (Label 0)
🪟 Rows 1501-1560 → Skipped (Label 0)
🪟 Ro